In [ ]:
import pandas as pd
import numpy as np
from pymol import cmd
from Bio import SeqIO
from Bio import Align,AlignIO
from Bio.PDB import PDBParser
from Bio.PDB.Polypeptide import standard_aa_names
import re
from modeller import *              # Load standard Modeller classes
from modeller.automodel import *    # Load the AutoModel class
from modeller.scripts import complete_pdb
import os

In proteinfolder, the original pdb file and fasta file from protein data bank are required before running the following code
the pdb file has name xxxx.pdb (e.g., 1t5z.pdb)
the fasta file has name xxxx_1.fasta (e.g., 1t5z_1.fasta) (assuming the protein sequence is always 1)

In [ ]:
proteinfolder = Path(Your_proteinfolder)
proteinlist = [A List of PDB ID] #(e.g., ['2PIP','2PIO'])

In [ ]:
def combine_fastanpdb(proteinid):
    seqactual = ''
    foundname = ''
    for record in SeqIO.parse(proteinfolder/f'{proteinid}.pdb', "pdb-atom"):
        name = re.sub('[^a-zA-Z0-9]', '_', record.id)
        if len(str(record.seq)) > len(seqactual):
            seqactual = str(record.seq)
            seqactual = seqactual.replace('X','')
            foundname = name

    records = SeqIO.parse(proteinfolder/f"{proteinid.upper()}_1.fasta", "fasta")
    for record in records:
        nameref = foundname+'ref'
        seqref = str(record.seq)

    pdbid = foundname.split('_')[0]
    with open(proteinfolder/f'{pdbid}.fasta', 'w') as f:
        f.write(f"> {pdbid} \n")
        f.write(seqref+'\n')    
        f.write(f"> {foundname} \n")
        f.write(seqactual+'\n')
    f.close()
    return foundname,seqref,seqactual

In [ ]:
def align2pir(name,seqref,seqactual):
    pdbid = name.split('_')[0]
    aligner = Align.PairwiseAligner()
    alignments = aligner.align(seqref, seqactual)
    aligresults = format(alignments[0],'fasta')
    aligresults = aligresults.split('\n')
    aligresults[0] = f'> {pdbid}'
    aligresults[2] = f'> {name}'
    aligresults = [i+'\n' for i in aligresults]
    with open(proteinfolder/f'{pdbid}.fasta', 'w') as f:
        f.writelines(aligresults)
    f.close()

    records = SeqIO.parse(proteinfolder/f"{pdbid}.fasta", "fasta")
    count = SeqIO.write(records, proteinfolder/f"{pdbid}.pir", "pir")
    alig_coord = alignments[0].coordinates
    return alig_coord

In [ ]:
def pir2modellerpir(name,alig_coord):
    pdbid = name.split('_')[0]
    with open(proteinfolder/f'{pdbid}.pir', 'r') as pir:
        pirlines = pir.readlines()
    pir.close() 

    parser = PDBParser()
    structure = parser.get_structure(f'{pdbid.lower()}', proteinfolder/f'{pdbid.lower()}.pdb')
    structureinfo = {}
    sequencelength = 0
    for model in structure:
        for chain in model:
            protein_residues = [res for res in chain if res.get_resname() in standard_aa_names]
            head_res = protein_residues[0].get_id()[1]
            tail_res = protein_residues[-1].get_id()[1]
            if sequencelength < (tail_res-head_res):
                sequencelength = tail_res-head_res
                structureinfo['chainID'] = chain.id
                structureinfo['headres'] = head_res
                structureinfo['tailres'] = tail_res
    sequencehead = structureinfo['headres'] - (alig_coord[0][1]-alig_coord[1][1])
    sequencetail = structureinfo['tailres'] + (alig_coord[0][-1]-alig_coord[1][-1])
    namelist = []
    headerlist = []
    for index,line in enumerate(pirlines):
        if '>' in line:
            namelist.append(line.split(';')[1].replace('\n',''))
            pirlines[index] = pirlines[index].replace('XX','P1')
            headerlist.append(index+1)
    for name,index in zip(namelist,headerlist):
        if '_' in name:
            field1 = 'structureX'
            field3 = structureinfo['headres']
            field4 = structureinfo['chainID']
            field5 = structureinfo['tailres']
            field6 = structureinfo['chainID']
        else:
            field1 = 'sequence'
            field3 = sequencehead
            field4 = structureinfo['chainID']
            field5 = sequencetail
            field6 = structureinfo['chainID']
        field2 = name
        field7 = ' '
        field8 = ' '
        field9 = ' '
        field10 = ' '
        fieldlist = [field1,field2,field3,field4,field5,field6,field7,field8,field9,field10]
        fieldlist = [str(i) for i in fieldlist]
        newline = (':').join(fieldlist)
        pirlines[index] = newline+'\n'
    with open(proteinfolder/f'{pdbid}.ali', 'w') as f:
        f.writelines(pirlines)
    f.close()    
    shutil.copyfile(proteinfolder/f'{pdbid.lower()}.pdb',proteinfolder/f'{name}.pdb')

In [ ]:
def modeller_exe(name):
    pdbid = name.split('_')[0]
    log.verbose()    # request verbose output
    env = Environ()  # create a new MODELLER environment to build this model in    
    
    env.io.atom_files_directory = [str(proteinfolder)]

    a = AutoModel(env,
                alnfile  = proteinfolder/f"{pdbid}.ali",     # alignment filename
                knowns   = f'{name}',              # codes of the templates
                sequence = f'{pdbid}',              # code of the target

    )
    a.starting_model= 1                 # index of the first model
    a.ending_model  = 5                 # index of the last model
                                        # (determines how many models to calculate)
    a.make()                                            # do comparative modeling  
    scorelist = []
    for i in range(1,6,1):
        mdl = complete_pdb(env, f'{pdbid}.B9999000{str(i)}.pdb')

        # Assess all atoms with DOPE:
        s = Selection(mdl)
        score = s.assess_dope()
        scorelist.append(score)
    refpdb = proteinfolder/f"{pdbid.lower()}.pdb"
    current_directory = os.getcwd()
    current_directory = current_directory.replace("\\","/")
    selectstructure = str(1+np.argmin(scorelist))
    shutil.move(f"{current_directory}/{pdbid}.B9999000{selectstructure}.pdb",proteinfolder/f"{pdbid}.B9999000{selectstructure}.pdb")
    bestpdb = proteinfolder/f"{pdbid}.B9999000{selectstructure}.pdb"
    cmd.load(refpdb)
    cmd.load(bestpdb)
    cmd.align(f'{pdbid}.B9999000{selectstructure}',f'{pdbid.lower()}')
    cmd.save(proteinfolder/f"{name.replace('_','')}.pdb",selection=f'{pdbid}.B9999000{selectstructure}',format='pdb')
    cmd.delete('all')  

In [ ]:

for pro in proteinlist:
    proteinid = pro.lower()
    name,seqref,seqactual = combine_fastanpdb(proteinid)
    if not os.path.exists(proteinfolder/f"{name.replace('_','')}.pdb"):
        alig_coord = align2pir(name,seqref,seqactual)
        pir2modellerpir(name,alig_coord)
        modeller_exe(name)    
        print(f'{pro} is done')